In [ ]:
# load init moudle file
require 'erb'

load "/work/pannote/panbot_note.rb"
nil

In [ ]:
# March - 49058	
# Feb - 41293-49057
# Jan -  32630-41292

# month = [[32630,41292],[41293,49057],[49058,54632]]

month = [[54632-288*14,54632]]

month_epoch = month.map { |x|
    Epoch.where(" ? <= epoch and epoch <= ? ",x[0],x[1])
} 

[#<ActiveRecord::Relation [#<Epoch id: 26862, epoch: 50600, start_timestamp: "2022-03-06 12:21:44.000000000 +0000", lock_timestamp: "2022-03-06 12:26:44.000000000 +0000", close_timestamp: "2022-03-06 12:31:53.000000000 +0000", lock_price: 375.75244614, close_price: 376.44859476, total_amount: 32.43360382629296, bull_amount: 16.455563137283757, bear_amount: 15.978040689009203, reward_base_cal_amount: 16.455563137283757, reward_amount: 31.460595711504173, bull_payout: 1.9709810934885224, bear_payout: 2.029886170499179, created_at: "2022-03-06 13:00:42.097423000 +0000", updated_at: "2022-03-06 13:01:36.885296000 +0000", total_count: 161, start_block_number: 15822508, lock_block_number: 15822608>, #<Epoch id: 26861, epoch: 50601, start_timestamp: "2022-03-06 12:26:53.000000000 +0000", lock_timestamp: "2022-03-06 12:31:53.000000000 +0000", close_timestamp: "2022-03-06 12:37:02.000000000 +0000", lock_price: 376.44859476, close_price: 376.57104041, total_amount: 28.521905445015783, bull_amoun

In [ ]:
## 观测局面 -> 最终payout
## 观测局面 -> 最终win
## 最终payout -> 最终win

month_epoch = month_epoch.map do |epoch| 
    epoch = Parallel.map(epoch, in_threads: 10) {|x| 
    if x.block.last!=nil then
        last_block_number = x.block.last.block_number

        last_block_number = last_block_number + x.get_last_block_order(last_block_number) -1
        
        winner = "bear" if x.lock_price > x.close_price
        winner = "bull" if x.lock_price < x.close_price
        winner = "draw" if x.lock_price == x.close_price

        {  
            winner: winner,
            lock_price: x.lock_price,
            close_price: x.close_price,

            final:[
                x.get_bull_payout(last_block_number),
                x.get_bear_payout(last_block_number),
                x.get_bull_amount(last_block_number),
                x.get_bear_amount(last_block_number),  
            ],
            ob:[
                x.get_bull_payout(last_block_number-2),
                x.get_bear_payout(last_block_number-2),
                x.get_bull_amount(last_block_number-2),
                x.get_bear_amount(last_block_number-2),  
            ]
        }
    else


    end
    }
    epoch = epoch.filter {|x| x!=nil}
end

nil

In [ ]:
month_epoch = [month_epoch.flatten]

[[{:winner=>"bull", :lock_price=>375.75244614, :close_price=>376.44859476, :final=>[1.9709810934885224, 2.0298861704991795, 16.455563137283757, 15.978040689009202], :ob=>[4.091174667692816, 1.323501615891016, 4.954304677055601, 15.314621113746313]}, {:winner=>"bull", :lock_price=>376.44859476, :close_price=>376.57104041, :final=>[1.7274657924023873, 2.374635083111735, 16.510836608434573, 12.011068836581204], :ob=>[2.509151341743686, 1.6626240671426573, 6.6283361962994745, 10.003162464173593]}, {:winner=>"bear", :lock_price=>376.57104041, :close_price=>375.67666837, :final=>[2.1393690180285727, 1.8776787714750043, 14.139951669252946, 16.11062284836821], :ob=>[2.052110803335104, 1.950470232631471, 11.280278090646622, 11.86810244379359]}, {:winner=>"bear", :lock_price=>375.67666837, :close_price=>375.49856737, :final=>[2.1998573534329013, 1.8334324052262616, 8.422532835210674, 10.105837956857593], :ob=>[1.8321808146932692, 2.2016619253215866, 7.242702274455049, 6.027237879336797]}, {:winn

In [ ]:
def avg(arr)
    arr = arr.filter {|x| x>-999 }

    arr.sum / arr.count.to_f

end


:avg

In [ ]:
## 最终payout -> 最终win

## payout二分 (>2, <2)

IRuby.html "<h1> 最终payout - 最终win </h1>"+(month_epoch.map.with_index do |epoch,i|

    group = []
    group.push ["bull_payout >2",epoch.filter {|x| x[:final][0]>2}]
    group.push ["bear_payout >2",epoch.filter {|x| x[:final][1]>2}]

    group = group.map {|x| 
        {
            group:x[0],
            count:x[1].count,
            bull:x[1].filter{|x| x[:winner]=="bull"}.count,
            bear:x[1].filter{|x| x[:winner]=="bear"}.count,
            draw:x[1].filter{|x| x[:winner]=="draw"}.count,
            bull_win_ratio: x[1].filter{|x| x[:winner]=="bull"}.count / x[1].count.to_f,
            bear_win_ratio: x[1].filter{|x| x[:winner]=="bear"}.count / x[1].count.to_f,
            bull_win_price: avg(x[1].filter{|x| x[:winner]=="bull"}.map {|x| x[:close_price]/x[:lock_price] - 1}),
            bear_win_price: avg(x[1].filter{|x| x[:winner]=="bear"}.map {|x| x[:close_price]/x[:lock_price] - 1}),
            avg_price: avg( x[1].map {|x| x[:close_price] / x[:lock_price] -1 } ),
        }
    }


    html = """
    <h4>Month <%= i+1 %></h4>
    <table>
    <tr>
    <th>group</th>
    <th>count</th>
    <th>bull</th>
    <th>bear</th>
    <th>draw</th>
    <th>bull_win</th>
    <th>bear_win</th>
    <th>bull_win</th>
    <th>bear_win</th>
    <th>avg_price</th>
    </tr>
    <% group.each do |x| %>
        <tr>
    <td><%= x[:group] %></td>
    <td><%= x[:count] %></td>
    <td><%= x[:bull] %></td>
    <td><%= x[:bear] %></td>
    <td><%= x[:draw] %></td>
    <td><%= (x[:bull_win_ratio]*100).round(4) %>%</td>
    <td><%= (x[:bear_win_ratio]*100).round(4) %>%</td>
    <td><%= (x[:bull_win_price]*100).round(4) %>%</td>
    <td><%= (x[:bear_win_price]*100).round(4) %>%</td>
    <td><%= (x[:avg_price]*100).round(4) %>%</td>
    </tr>
    <% end %>
    </table>
    """

   ERB.new(html).result(binding)
end).join


group,count,bull,bear,draw,bull_win,bear_win,bull_win,bear_win,avg_price
bull_payout >2,1755,785,963,7,44.7293%,54.8718%,0.096%,-0.1264%,-0.0264%
bear_payout >2,2270,1270,994,6,55.9471%,43.7885%,0.1197%,-0.1017%,0.0224%


In [ ]:
## 最终payout -> 最终win

## payout二分 (>2, <2)

IRuby.html "<h1> 最终payout - 最终win </h1>"+(month_epoch.map.with_index do |epoch,i|

    group = []
    group.push ["bull_payout 2.0-2.1",epoch.filter {|x| 2<x[:final][0] and x[:final][0]<=2.1}]
    group.push ["bull_payout 2.1-2.2",epoch.filter {|x| 2.1<x[:final][0] and x[:final][0]<=2.2}]
    group.push ["bull_payout 2.2-2.3",epoch.filter {|x| 2.2<x[:final][0] and x[:final][0]<=2.3}]
    group.push ["bull_payout 2.3-2.4",epoch.filter {|x| 2.3<x[:final][0] and x[:final][0]<=2.4}]
    group.push ["bull_payout 2.4-2.5",epoch.filter {|x| 2.4<x[:final][0] and x[:final][0]<=2.5}]
    group.push ["bull_payout 2.5-3.0",epoch.filter {|x| 2.5<x[:final][0] and x[:final][0]<=3}]
    group.push ["bull_payout 3.0-3.5",epoch.filter {|x| 3<x[:final][0] and x[:final][0]<=3.5}]
    group.push ["bull_payout 3.5-4.0",epoch.filter {|x| 3.5<x[:final][0] and x[:final][0]<=4}]
    group.push ["bull_payout 4.0-inf",epoch.filter {|x| 4<x[:final][0] and x[:final][0]<=999}]
    group.push ["bear_payout 2.0-2.1",epoch.filter {|x| 2<x[:final][1] and x[:final][1]<=2.1}]
    group.push ["bear_payout 2.1-2.2",epoch.filter {|x| 2.1<x[:final][1] and x[:final][1]<=2.2}]
    group.push ["bear_payout 2.2-2.3",epoch.filter {|x| 2.2<x[:final][1] and x[:final][1]<=2.3}]
    group.push ["bear_payout 2.3-2.4",epoch.filter {|x| 2.3<x[:final][1] and x[:final][1]<=2.4}]
    group.push ["bear_payout 2.4-2.5",epoch.filter {|x| 2.4<x[:final][1] and x[:final][1]<=2.5}]
    group.push ["bear_payout 2.5-3.0",epoch.filter {|x| 2.5<x[:final][1] and x[:final][1]<=3}]
    group.push ["bear_payout 3.0-3.5",epoch.filter {|x| 3<x[:final][1] and x[:final][1]<=3.5}]
    group.push ["bear_payout 3.5-4.0",epoch.filter {|x| 3.5<x[:final][1] and x[:final][1]<=4}]
    group.push ["bear_payout 4.0-inf",epoch.filter {|x| 4<x[:final][1] and x[:final][1]<=999}]

    group = group.map {|x| 
        {
            group:x[0],
            count:x[1].count,
            bull:x[1].filter{|x| x[:winner]=="bull"}.count,
            bear:x[1].filter{|x| x[:winner]=="bear"}.count,
            draw:x[1].filter{|x| x[:winner]=="draw"}.count,
            bull_win_ratio: x[1].filter{|x| x[:winner]=="bull"}.count / x[1].count.to_f,
            bear_win_ratio: x[1].filter{|x| x[:winner]=="bear"}.count / x[1].count.to_f,
            bull_win_price: avg(x[1].filter{|x| x[:winner]=="bull"}.map {|x| x[:close_price]/x[:lock_price] - 1}),
            bear_win_price: avg(x[1].filter{|x| x[:winner]=="bear"}.map {|x| x[:close_price]/x[:lock_price] - 1}),
            avg_price: avg(x[1].map {|x| x[:close_price]/x[:lock_price] - 1}),

        }
    }


    html = """
    <h4>Month <%= i+1 %></h4>
    <table>
    <tr>
    <th>group</th>
    <th>count</th>
    <th>bull</th>
    <th>bear</th>
    <th>draw</th>
    <th>bull_win</th>
    <th>bear_win</th>
    <th>bull_win</th>
    <th>bear_win</th>
    <th>avg_price</th>
    </tr>
    <% group.each do |x| %>
        <tr>
    <td><%= x[:group] %></td>
    <td><%= x[:count] %></td>
    <td><%= x[:bull] %></td>
    <td><%= x[:bear] %></td>
    <td><%= x[:draw] %></td>
    <td><%= (x[:bull_win_ratio]*100).round(4) %>%</td>
    <td><%= (x[:bear_win_ratio]*100).round(4) %>%</td>
    <td><%= (x[:bull_win_price]*100).round(4) %>%</td>
    <td><%= (x[:bear_win_price]*100).round(4) %>%</td>
    <td><%= (x[:avg_price]*100).round(4) %>%</td>
    </tr>
    <% end %>
    </table>
    """

   ERB.new(html).result(binding)
end).join


group,count,bull,bear,draw,bull_win,bear_win,bull_win,bear_win,avg_price
bull_payout 2.0-2.1,438,226,209,3,51.5982%,47.7169%,0.0918%,-0.115%,-0.0075%
bull_payout 2.1-2.2,341,156,185,0,45.7478%,54.2522%,0.0993%,-0.1158%,-0.0174%
bull_payout 2.2-2.3,289,125,163,1,43.2526%,56.4014%,0.0878%,-0.1143%,-0.0265%
bull_payout 2.3-2.4,215,91,123,1,42.3256%,57.2093%,0.0931%,-0.1146%,-0.0262%
bull_payout 2.4-2.5,137,69,68,0,50.365%,49.635%,0.1113%,-0.1384%,-0.0127%
bull_payout 2.5-3.0,251,88,161,2,35.0598%,64.1434%,0.0993%,-0.129%,-0.0479%
bull_payout 3.0-3.5,54,25,29,0,46.2963%,53.7037%,0.1124%,-0.217%,-0.0645%
bull_payout 3.5-4.0,15,4,11,0,26.6667%,73.3333%,0.0643%,-0.237%,-0.1566%
bull_payout 4.0-inf,15,1,14,0,6.6667%,93.3333%,0.1949%,-0.3189%,-0.2846%
bear_payout 2.0-2.1,430,217,213,0,50.4651%,49.5349%,0.1059%,-0.1073%,0.0003%


In [ ]:
## 最终payout -> 最终win

## ob payout二分 (>2, <2)

IRuby.html "<h1> ob payout - 最终 </h1>"+(month_epoch.map.with_index do |epoch,i|

    group = []
    group.push ["bull_payout >2",epoch.filter {|x| x[:ob][0]>2}]
    group.push ["bear_payout >2",epoch.filter {|x| x[:ob][1]>2}]

    group = group.map {|x| 
        {
            group:x[0],
            bull:x[1].filter{|x| x[:winner]=="bull"}.count,
            bear:x[1].filter{|x| x[:winner]=="bear"}.count,
            draw:x[1].filter{|x| x[:winner]=="draw"}.count,
            bull_win_ratio: x[1].filter{|x| x[:winner]=="bull"}.count / x[1].count.to_f,
            bear_win_ratio: x[1].filter{|x| x[:winner]=="bear"}.count / x[1].count.to_f,
            final_bull_payout: x[1].map {|x| x[:final][0]}.sum / x[1].count.to_f,
            final_bear_payout: x[1].map {|x| x[:final][1]}.sum / x[1].count.to_f,
        }
    }


    html = """
    <h4>Month <%= i+1 %></h4>
    <table>
    <tr>
    <th>group</th>
    <th>bull</th>
    <th>bear</th>
    <th>draw</th>
    <th>bull_win_ratio</th>
    <th>bear_win_ratio</th>
    <th>final_bull_payout</th>
    <th>final_bear_payout</th>
    </tr>
    <% group.each do |x| %>
        <tr>
    <td><%= x[:group] %></td>
    <td><%= x[:bull] %></td>
    <td><%= x[:bear] %></td>
    <td><%= x[:draw] %></td>
    <td><%= x[:bull_win_ratio].round(4) %></td>
    <td><%= x[:bear_win_ratio].round(4) %></td>
    <td><%= x[:final_bull_payout].round(4) %></td>
    <td><%= x[:final_bear_payout].round(4) %></td>
    </tr>
    <% end %>
    </table>
    """

   ERB.new(html).result(binding)
end).join


group,bull,bear,draw,bull_win_ratio,bear_win_ratio,final_bull_payout,final_bear_payout
bull_payout >2,4533,5035,33,0.4721,0.5244,2.2762,1.8847
bear_payout >2,6571,5764,44,0.5308,0.4656,1.878,2.2747


In [ ]:
## 最终payout -> 最终win

## payout二分 (>2, <2)

IRuby.html "<h1> ob payout - 最终win </h1>"+(month_epoch.map.with_index do |epoch,i|

    group = []
    group.push ["bull_payout 2.0-2.1",epoch.filter {|x| 2<x[:ob][0] and x[:ob][0]<=2.1}]
    group.push ["bull_payout 2.1-2.2",epoch.filter {|x| 2.1<x[:ob][0] and x[:ob][0]<=2.2}]
    group.push ["bull_payout 2.2-2.3",epoch.filter {|x| 2.2<x[:ob][0] and x[:ob][0]<=2.3}]
    group.push ["bull_payout 2.3-2.4",epoch.filter {|x| 2.3<x[:ob][0] and x[:ob][0]<=2.4}]
    group.push ["bull_payout 2.4-2.5",epoch.filter {|x| 2.4<x[:ob][0] and x[:ob][0]<=2.5}]
    group.push ["bull_payout 2.5-3.0",epoch.filter {|x| 2.5<x[:ob][0] and x[:ob][0]<=3}]
    group.push ["bull_payout 3.0-3.5",epoch.filter {|x| 3<x[:ob][0] and x[:ob][0]<=3.5}]
    group.push ["bull_payout 3.5-4.0",epoch.filter {|x| 3.5<x[:ob][0] and x[:ob][0]<=4}]
    group.push ["bull_payout 4.0-inf",epoch.filter {|x| 4<x[:ob][0] and x[:ob][0]<=999}]
    group.push ["bear_payout 2.0-2.1",epoch.filter {|x| 2<x[:ob][1] and x[:ob][1]<=2.1}]
    group.push ["bear_payout 2.1-2.2",epoch.filter {|x| 2.1<x[:ob][1] and x[:ob][1]<=2.2}]
    group.push ["bear_payout 2.2-2.3",epoch.filter {|x| 2.2<x[:ob][1] and x[:ob][1]<=2.3}]
    group.push ["bear_payout 2.3-2.4",epoch.filter {|x| 2.3<x[:ob][1] and x[:ob][1]<=2.4}]
    group.push ["bear_payout 2.4-2.5",epoch.filter {|x| 2.4<x[:ob][1] and x[:ob][1]<=2.5}]
    group.push ["bear_payout 2.5-3.0",epoch.filter {|x| 2.5<x[:ob][1] and x[:ob][1]<=3}]
    group.push ["bear_payout 3.0-3.5",epoch.filter {|x| 3<x[:ob][1] and x[:ob][1]<=3.5}]
    group.push ["bear_payout 3.5-4.0",epoch.filter {|x| 3.5<x[:ob][1] and x[:ob][1]<=4}]
    group.push ["bear_payout 4.0-inf",epoch.filter {|x| 4<x[:ob][1] and x[:ob][1]<=999}]

    group = group.map {|x| 
        {
            group:x[0],
            bull:x[1].filter{|x| x[:winner]=="bull"}.count,
            bear:x[1].filter{|x| x[:winner]=="bear"}.count,
            draw:x[1].filter{|x| x[:winner]=="draw"}.count,
            bull_win_ratio: x[1].filter{|x| x[:winner]=="bull"}.count / x[1].count.to_f,
            bear_win_ratio: x[1].filter{|x| x[:winner]=="bear"}.count / x[1].count.to_f,
            final_bull_payout: x[1].map {|x| x[:final][0]}.sum / x[1].count.to_f,
            final_bear_payout: x[1].map {|x| x[:final][1]}.sum / x[1].count.to_f,
        }
    }


    html = """
    <h4>Month <%= i+1 %></h4>
    <table>
    <tr>
    <th>group</th>
    <th>bull</th>
    <th>bear</th>
    <th>draw</th>
    <th>bull_win_ratio</th>
    <th>bear_win_ratio</th>
    <th>final_bull_payout</th>
    <th>final_bear_payout</th>
    </tr>
    <% group.each do |x| %>
        <tr>
    <td><%= x[:group] %></td>
    <td><%= x[:bull] %></td>
    <td><%= x[:bear] %></td>
    <td><%= x[:draw] %></td>
    <td><%= x[:bull_win_ratio] %></td>
    <td><%= x[:bear_win_ratio] %></td>
    <td><%= x[:final_bull_payout].round(4) %></td>
    <td><%= x[:final_bear_payout].round(4) %></td>
    </tr>
    <% end %>
    </table>
    """

   ERB.new(html).result(binding)
end).join


group,bull,bear,draw,bull_win_ratio,bear_win_ratio,final_bull_payout,final_bear_payout
bull_payout 2.0-2.1,530,570,5,0.4796380090497738,0.5158371040723982,2.0638,2.023
bull_payout 2.1-2.2,453,505,3,0.4713839750260146,0.5254942767950052,2.0946,1.9936
bull_payout 2.2-2.3,475,452,2,0.511302475780409,0.4865446716899892,2.132,1.9654
bull_payout 2.3-2.4,379,403,3,0.48280254777070064,0.513375796178344,2.1647,1.9353
bull_payout 2.4-2.5,318,310,3,0.5039619651347068,0.49128367670364503,2.1798,1.9216
bull_payout 2.5-3.0,1081,1199,6,0.47287839020122485,0.5244969378827646,2.228,1.8928
bull_payout 3.0-3.5,548,613,5,0.4699828473413379,0.5257289879931389,2.3292,1.8252
bull_payout 3.5-4.0,268,365,2,0.4220472440944882,0.5748031496062992,2.4405,1.769
bull_payout 4.0-inf,481,618,4,0.43608340888485947,0.5602901178603807,2.8524,1.6393
bear_payout 2.0-2.1,541,588,4,0.47749338040600176,0.5189761694616064,2.0215,2.0737


In [ ]:
epoch = Epoch.find_by_epoch(53560)
epoch.get_wrong_bet("betBear",16125802);

false

In [ ]:
epoch.get_detail(epoch.lock_block_number).attributes


{"id"=>3394927, "epoch"=>53560, "block_number"=>16125804, "bet_count"=>188, "bull_amount"=>17.656768503660757, "bear_amount"=>20.782666436128817, "bull_payout"=>2.1770368078292455, "bear_payout"=>1.8495911031399723, "block_time"=>2022-03-17 02:20:22 UTC, "created_at"=>2022-03-17 04:31:01.673151 UTC, "updated_at"=>2022-03-17 04:31:01.673151 UTC}

In [ ]:
block_number = 16125802
block_number = epoch.lock_block_number if block_number>epoch.lock_block_number
block_number
detail = epoch.get_detail(block_number)

#<EpochDetail id: 3394906, epoch: 53560, block_number: 16125802, bet_count: 157, bull_amount: 17.43066594044281, bear_amount: 11.977442779262399, bull_payout: 1.6871477441072527, bear_payout: 2.455291105261806, block_time: "2022-03-17 02:20:16.000000000 +0000", created_at: "2022-03-17 04:31:01.632307000 +0000", updated_at: "2022-03-17 04:31:01.632307000 +0000">

In [ ]:
Cache.find(3200)

#<Cache id: 3200, key: "PayoutBot{\"min_amount\":20,\"min_payout\":2.4,\"bet_am...", value_type: "JSON", value_s: "[{\"epoch\":32630,\"bet\":\"\",\"amount\":null,\"bet_block_...", value_i: nil, value_f: nil, value_t: nil, value_b: nil, created_at: "2022-03-06 17:55:48.001115000 +0000", updated_at: "2022-03-06 17:55:48.001115000 +0000">

In [ ]:
Cache.where("key like ?","%bot%").map {|x|x.key}

["bot_stats_calc.panbot_address", "bot_stats_calc.addr_list"]

In [ ]:
Cache.get("bot_stats_calc.addr_list").filter {|x| x[0]=="0xd3535d1b78ee416bf9789d9b9160ea4f2c6b3c1a"}

[["0xd3535d1b78ee416bf9789d9b9160ea4f2c6b3c1a", 230, 161]]

In [ ]:
Epoch.find_by_epoch(51566).lock_block_number

15921477

In [ ]:
epoch = Epoch.find_by_epoch(51566)

#<Epoch id: 27833, epoch: 51566, start_timestamp: "2022-03-09 23:14:09.000000000 +0000", lock_timestamp: "2022-03-09 23:19:09.000000000 +0000", close_timestamp: "2022-03-09 23:24:18.000000000 +0000", lock_price: 393.0540816, close_price: 392.98682684, total_amount: 42.25460637260137, bull_amount: 18.266838283545564, bear_amount: 23.98776808905581, reward_base_cal_amount: 23.98776808905581, reward_amount: 40.986968181423336, bull_payout: 2.313186645477863, bear_payout: 1.7615063734036862, created_at: "2022-03-17 03:40:26.797281000 +0000", updated_at: "2022-03-17 04:23:08.435286000 +0000", total_count: 153, start_block_number: 15921377, lock_block_number: 15921477>

In [ ]:
tx = Tx.where(from:"0xc15c6d0a69c384fac445a4fa3d45f0dcd8d39a9d")
  .where("? <= block_number and block_number <= ?",Epoch.find_by_epoch(51566).start_block_number,Epoch.find_by_epoch(51566).lock_block_number)
  .where("method_name='betBull' or method_name='betBear' ")
  .where(tx_status:true).first

#<Tx id: 7009345, block_hash: "0x89aab26c2f1d2cdbc58946b8ab41da50e128f16c40a36fdb...", tx_hash: "0x72cae21394aba453e031888521723de959d603b9cc575ed7...", from: "0xc15c6d0a69c384fac445a4fa3d45f0dcd8d39a9d", to: "0x18b2a687610328590bc8f2e5fedde3b582a49cda", method_name: "betBull", input_data: "[51566]", contract_address: nil, tx_type: 0, tx_status: true, block_time: "2022-03-09 23:19:00.000000000 +0000", block_number: 15921474, gas: 98820, gas_price: 0, nonce: 15179, tx_index: 18, amount: 300000000000000000, cumulative_gas_used: 1856854, gas_used: 93132, created_at: "2022-03-17 03:16:37.148037000 +0000", updated_at: "2022-03-17 03:16:37.148037000 +0000">

In [ ]:
epoch.get_bull_amount(15921474)

7.132242306417596

In [ ]:
tx.amount/1e18

0.3

In [ ]:
nil-1

NoMethodError: undefined method `-' for nil:NilClass

In [ ]:
Vault.all.map { |x| x.key }

["pancake_prediction_v2", "bsc_endpoint", "pancake_prediction_v2.abi", "bscscan-apikey", "aws_pem_file", "bot_private_key"]

In [ ]:
Vault.set("bot_address",
Vault.get("bot_private_key").map do |pkey|
    (Eth::Key.new priv: pkey).address.downcase
end)

["0xacc961474eb17c103bbed96fc8f0e4eeac91c475", "0x6cf545a3bf58e59b0b16f88ce0b3e6d6589c7b80", "0x65fcb37a28d8504b9dd87ddabe4e0302ec247d63"]

In [ ]:
Vault.get("bot_address")

["0xacc961474eb17c103bbed96fc8f0e4eeac91c475", "0x6cf545a3bf58e59b0b16f88ce0b3e6d6589c7b80", "0x65fcb37a28d8504b9dd87ddabe4e0302ec247d63"]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>